In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sanity").getOrCreate()

25/09/19 14:12:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
df=spark.range(5)
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [3]:
spark.sql("SHOW CATALOGS").show(truncate=False)

+-------------+
|catalog      |
+-------------+
|demo         |
|spark_catalog|
+-------------+



In [13]:
print(spark.version)
print(spark.sparkContext.appName)
spark.sparkContext.getConf().getAll()


3.5.5
PySparkShell


[('spark.eventLog.enabled', 'true'),
 ('spark.app.submitTime', '1758225982978'),
 ('spark.driver.port', '44679'),
 ('spark.history.fs.logDirectory', '/home/iceberg/spark-events'),
 ('spark.sql.warehouse.dir',
  'file:/home/iceberg/notebooks/notebooks/spark-warehouse'),
 ('spark.sql.catalog.demo.s3.endpoint', 'http://minio:9000'),
 ('spark.eventLog.dir', '/home/iceberg/spark-events'),
 ('spark.app.startTime', '1758225983187'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/ja

## Running a small test with sample data


In [3]:
spark.stop()


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("df-basics").getOrCreate()


25/09/19 14:12:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
spark.sparkContext.appName

'df-basics'

In [6]:
orders = spark.read.option("header", True).option("inferSchema", True).csv("/home/iceberg/data/testing-spark/orders.csv")
customers = spark.read.option("header", True).option("inferSchema", True).csv("/home/iceberg/data/testing-spark/customers.csv")
products = spark.read.option("header", True).option("inferSchema", True).csv("/home/iceberg/data/testing-spark/products.csv")


In [7]:

orders.printSchema(); customers.printSchema(); products.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_ts: timestamp (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- qty: integer (nullable = true)

root
 |-- customer_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- signup_date: date (nullable = true)

root
 |-- product_id: integer (nullable = true)
 |-- sku: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: double (nullable = true)



In [8]:
from pyspark.sql import functions as F

orders = (orders
  .withColumn("order_date", F.to_date("order_ts"))
        )

              
bg_customers = customers.filter(F.col("country") == "BG").select("customer_id","country")
bg_customers.show()

+-----------+-------+
|customer_id|country|
+-----------+-------+
|          1|     BG|
|          2|     BG|
+-----------+-------+



In [9]:
line_items = orders\
    .join(products, "product_id")\
    .withColumn("line_amount", F.col("qty")*F.col("price"))
               
daily_sales = line_items\
            .groupBy("order_date")\
            .agg(
                F.sum("line_amount").alias("gross_sales"),
                F.countDistinct("order_id").alias("orders"),
                F.sum("qty").alias("units")
            )\
            .orderBy("order_date")

daily_sales.show()

+----------+-----------+------+-----+
|order_date|gross_sales|orders|units|
+----------+-----------+------+-----+
|2025-03-21|      47.48|     2|    3|
|2025-03-22|      84.99|     3|    5|
|2025-03-23|      54.98|     2|    4|
|2025-03-24|       41.0|     2|    2|
|2025-03-25|      67.47|     3|    4|
|2025-03-26|       70.0|     2|    3|
|2025-03-27|      67.47|     3|    4|
|2025-03-28|       29.0|     1|    1|
+----------+-----------+------+-----+



In [51]:
daily_sales.repartition(1).write.mode("overwrite").partitionBy("order_date").parquet("data/testing-spark-output/daily_sales")

#daily_sales.repartition(F.col("order_date")).sortWithinPartitions("order_date").mode("overwrite").partitionBy("order_date").parquet("data/testing-spark-output/daily_sales")

In [10]:
ds = spark.read.parquet("data/testing-spark-output/daily_sales")
#ds.show()
ds.orderBy("order_date").show()

#Interesting is that first daily_sales is ordered by date but
#this seems to be done only for the benefit of the partitionBy 
#when writing to parquet. 
#Later on when I read the parquet it reads all the separate partitions but the "ds" dataframe is not ordered by date.

+-----------+------+-----+----------+
|gross_sales|orders|units|order_date|
+-----------+------+-----+----------+
|      47.48|     2|    3|2025-03-21|
|      84.99|     3|    5|2025-03-22|
|      54.98|     2|    4|2025-03-23|
|       41.0|     2|    2|2025-03-24|
|      67.47|     3|    4|2025-03-25|
|       70.0|     2|    3|2025-03-26|
|      67.47|     3|    4|2025-03-27|
|       29.0|     1|    1|2025-03-28|
+-----------+------+-----+----------+

